# Inference LLM

In [1]:
!sudo apt update
!sudo apt install -y ffmpeg

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [44.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]33m
Get:5 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [848 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]m
Get:7 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1792 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [823 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [977 kB]m
Get:10 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [266 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB]
Get:12 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]33m
Get:13 http://archive.ubuntu.com/ubunt

Get:20 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libfreetype6 amd64 2.11.1+dfsg-1ubuntu0.2 [389 kB]
Get:21 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1041 kB]
Get:22 http://archive.ubuntu.com/ubuntu jammy/main amd64 fontconfig-config all 2.13.1-4.2ubuntu5 [29.1 kB]
Get:23 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontconfig1 amd64 2.13.1-4.2ubuntu5 [131 kB]
Get:24 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpixman-1-0 amd64 0.40.0-1ubuntu0.22.04.1 [264 kB]
Get:25 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxcb-render0 amd64 1.14-3ubuntu3 [16.4 kB]
Get:26 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxcb-shm0 amd64 1.14-3ubuntu3 [5780 B]
Get:27 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxrender1 amd64 1:0.9.10-1build4 [19.7 kB]
Get:28 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcairo2 amd64 1.16.0-5ubuntu2 [628 kB]
Get:29 http://archive.ubuntu.com/ubuntu jammy/univer

Get:99 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcaca0 amd64 0.99.beta19-2.2ubuntu4 [224 kB]
Get:100 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcdio19 amd64 2.1.0-3build1 [63.3 kB]
Get:101 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcdio-cdda2 amd64 10.2+2.0.0-1build3 [16.7 kB]
Get:102 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcdio-paranoia2 amd64 10.2+2.0.0-1build3 [15.9 kB]
Get:103 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libdc1394-25 amd64 2.2.6-4 [88.8 kB]
Get:104 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglvnd0 amd64 1.4.0-1 [73.6 kB]
Get:105 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libglapi-mesa amd64 22.2.5-0ubuntu0.1~22.04.3 [35.5 kB]
Get:106 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxcb-dri2-0 amd64 1.14-3ubuntu3 [7206 B]
Get:107 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxcb-dri3-0 amd64 1.14-3ubuntu3 [6968 B]
Get:108 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxcb-

Selecting previously unselected package libxml2:amd64.
Preparing to unpack .../004-libxml2_2.9.13+dfsg-1ubuntu0.3_amd64.deb ...
Unpacking libxml2:amd64 (2.9.13+dfsg-1ubuntu0.3) ...
Selecting previously unselected package shared-mime-info.
Preparing to unpack .../005-shared-mime-info_2.1-2_amd64.deb ...
Unpacking shared-mime-info (2.1-2) ...
Selecting previously unselected package xkb-data.
Preparing to unpack .../006-xkb-data_2.33-1_all.deb ...
7Progress: [  2%] [#.........................................................] 8Unpacking xkb-data (2.33-1) ...
Selecting previously unselected package libdrm-common.
Preparing to unpack .../007-libdrm-common_2.4.113-2~ubuntu0.22.04.1_all.deb ...
Unpacking libdrm-common (2.4.113-2~ubuntu0.22.04.1) ...
Selecting previously unselected package libdrm2:amd64.
Preparing to unpack .../008-libdrm2_2.4.113-2~ubuntu0.22.04.1_amd64.deb ...
Unpacking libdrm2:amd64 (2.4.113-2~ubuntu0.22.04.1) ...
Selecting previously unselected package libusb-1.0-0:amd64.

Selecting previously unselected package libpango-1.0-0:amd64.
Preparing to unpack .../045-libpango-1.0-0_1.50.6+ds-2ubuntu1_amd64.deb ...
7Progress: [ 14%] [########..................................................] 8Unpacking libpango-1.0-0:amd64 (1.50.6+ds-2ubuntu1) ...
Selecting previously unselected package libpangoft2-1.0-0:amd64.
Preparing to unpack .../046-libpangoft2-1.0-0_1.50.6+ds-2ubuntu1_amd64.deb ...
Unpacking libpangoft2-1.0-0:amd64 (1.50.6+ds-2ubuntu1) ...
Selecting previously unselected package libpangocairo-1.0-0:amd64.
Preparing to unpack .../047-libpangocairo-1.0-0_1.50.6+ds-2ubuntu1_amd64.deb ...
Unpacking libpangocairo-1.0-0:amd64 (1.50.6+ds-2ubuntu1) ...
Selecting previously unselected package librsvg2-2:amd64.
Preparing to unpack .../048-librsvg2-2_2.52.5+dfsg-3ubuntu0.2_amd64.deb ...
Unpacking librsvg2-2:amd64 (2.52.5+dfsg-3ubuntu0.2) ...
7Progress: [ 15%] [########..................................................] 8Selecting previously unselected package 

Preparing to unpack .../085-libmysofa1_1.2.1~dfsg0-1_amd64.deb ...
Unpacking libmysofa1:amd64 (1.2.1~dfsg0-1) ...
Selecting previously unselected package libasyncns0:amd64.
Preparing to unpack .../086-libasyncns0_0.8-6build2_amd64.deb ...
Unpacking libasyncns0:amd64 (0.8-6build2) ...
Selecting previously unselected package libx11-xcb1:amd64.
Preparing to unpack .../087-libx11-xcb1_2%3a1.7.5-1ubuntu0.2_amd64.deb ...
Unpacking libx11-xcb1:amd64 (2:1.7.5-1ubuntu0.2) ...
7Progress: [ 27%] [###############...........................................] 8Selecting previously unselected package libpulse0:amd64.
Preparing to unpack .../088-libpulse0_1%3a15.99.1+dfsg1-1ubuntu2.1_amd64.deb ...
Unpacking libpulse0:amd64 (1:15.99.1+dfsg1-1ubuntu2.1) ...
Selecting previously unselected package libsphinxbase3:amd64.
Preparing to unpack .../089-libsphinxbase3_0.8+5prealpha+1-13build1_amd64.deb ...
Unpacking libsphinxbase3:amd64 (0.8+5prealpha+1-13build1) ...
Selecting previously unselected package lib

Selecting previously unselected package libjack-jackd2-0:amd64.
Preparing to unpack .../124-libjack-jackd2-0_1.9.20~dfsg-1_amd64.deb ...
Unpacking libjack-jackd2-0:amd64 (1.9.20~dfsg-1) ...
Selecting previously unselected package libopenal-data.
Preparing to unpack .../125-libopenal-data_1%3a1.19.1-2build3_all.deb ...
Unpacking libopenal-data (1:1.19.1-2build3) ...
Selecting previously unselected package libsndio7.0:amd64.
Preparing to unpack .../126-libsndio7.0_1.8.1-1.1_amd64.deb ...
Unpacking libsndio7.0:amd64 (1.8.1-1.1) ...
7Progress: [ 39%] [######################....................................] 8Selecting previously unselected package libopenal1:amd64.
Preparing to unpack .../127-libopenal1_1%3a1.19.1-2build3_amd64.deb ...
Unpacking libopenal1:amd64 (1:1.19.1-2build3) ...
Selecting previously unselected package libwayland-client0:amd64.
Preparing to unpack .../128-libwayland-client0_1.20.0-1ubuntu0.1_amd64.deb ...
Unpacking libwayland-client0:amd64 (1.20.0-1ubuntu0.1) ...

Setting up libgme0:amd64 (0.6.3-2) ...
7Progress: [ 50%] [#############################.............................] 8Setting up libssh-gcrypt-4:amd64 (0.9.6-2ubuntu0.22.04.1) ...
Setting up libgraphite2-3:amd64 (1.3.14-1build2) ...
Setting up libsrt1.4-gnutls:amd64 (1.4.4-4) ...
7Progress: [ 51%] [#############################.............................] 8Setting up libxcb-dri3-0:amd64 (1.14-3ubuntu3) ...
Setting up libpixman-1-0:amd64 (0.40.0-1ubuntu0.22.04.1) ...
Setting up libudfread0:amd64 (1.1.2-1) ...
7Progress: [ 52%] [##############################............................] 8Setting up libwayland-server0:amd64 (1.20.0-1ubuntu0.1) ...
Setting up libaom3:amd64 (3.3.0-1) ...
Setting up libx11-xcb1:amd64 (2:1.7.5-1ubuntu0.2) ...
7Progress: [ 53%] [##############################............................] 8Setting up libpciaccess0:amd64 (0.16-3) ...
Setting up librabbitmq4:amd64 (0.10.0-1ubuntu2) ...
Setting up libraw1394-11:amd64 (2.1.2-2build2) ...
Setting up libc

Setting up libcaca0:amd64 (0.99.beta19-2.2ubuntu4) ...
7Progress: [ 80%] [##############################################............] 8Setting up libpulse0:amd64 (1:15.99.1+dfsg1-1ubuntu2.1) ...
Setting up libcdio-cdda2:amd64 (10.2+2.0.0-1build3) ...
Setting up fontconfig-config (2.13.1-4.2ubuntu5) ...
7Progress: [ 81%] [###############################################...........] 8Setting up libcdio-paranoia2:amd64 (10.2+2.0.0-1build3) ...
Setting up libxcursor1:amd64 (1:1.2.0-2build4) ...
Setting up libopenal1:amd64 (1:1.19.1-2build3) ...
Setting up libthai0:amd64 (0.1.29-1build1) ...
7Progress: [ 82%] [###############################################...........] 8Setting up libdc1394-25:amd64 (2.2.6-4) ...
Setting up librubberband2:amd64 (2.0.0-2) ...
Setting up libjack-jackd2-0:amd64 (1.9.20~dfsg-1) ...
7Progress: [ 83%] [################################################..........] 8Setting up libdrm2:amd64 (2.4.113-2~ubuntu0.22.04.1) ...
Setting up libva-drm2:amd64 (2.14.0-1)

In [1]:
import os
import torch
from llama.llama_adapter import LLaMA_adapter
import util.misc as misc
from datasets import load_from_disk, load_dataset, Dataset, disable_caching
from llama import Tokenizer
import llama
from ImageBind.data import my_load_and_transform_audio_data
from pathlib import Path
from tqdm import tqdm
import json

disable_caching()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/u8915687/miniconda3/envs/imagebind_LLM/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/u8915687/miniconda3/envs/imagebind_LLM/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [2]:
llama_dir = "/home/u8915687/lab/big-superb/Macaw-LLM2/weights/llama/"
pretrained_path = "/home/u8915687/lab/big-superb/LLaMA-Adapter/imagebind_LLM/exp/first/checkpoint-3.pth"

model = llama.load(pretrained_path, llama_dir, knn=True, max_batch_size=16)
model.eval()

Loading LLaMA-Adapter from /home/u8915687/lab/big-superb/LLaMA-Adapter/imagebind_LLM/exp/first/checkpoint-3.pth
model args: ModelArgs(dim=4096, n_layers=32, n_heads=32, vocab_size=-1, multiple_of=256, norm_eps=1e-06, max_batch_size=16, max_seq_len=512, w_bias=True, w_lora=True, lora_rank=16)


LLaMA_adapter(
  (image_bind): ImageBindModel(
    (modality_preprocessors): ModuleDict(
      (vision): RGBDTPreprocessor(
        (cls_token): tensor((1, 1, 1280), requires_grad=False)
        
        (rgbt_stem): PatchEmbedGeneric(
          (proj): Sequential(
            (0): PadIm2Video()
            (1): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
          )
        )
        (pos_embedding_helper): SpatioTemporalPosEmbeddingHelper(
          (pos_embed): tensor((1, 257, 1280), requires_grad=False)
          
        )
      )
      (text): TextPreprocessor(
        (pos_embed): tensor((1, 77, 1024), requires_grad=False)
        (mask): tensor((77, 77), requires_grad=False)
        
        (token_embedding): Embedding(49408, 1024)
      )
      (audio): AudioPreprocessor(
        (cls_token): tensor((1, 1, 768), requires_grad=False)
        
        (rgbt_stem): PatchEmbedGeneric(
          (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 

In [5]:
tokenizer = Tokenizer(model_path="/home/u8915687/lab/big-superb/Macaw-LLM2/weights/llama_7B/tokenizer.model")

In [6]:
@torch.inference_mode()
def forward_inference(self, visual_feats, tokens, start_pos: int):
    _bsz, seqlen = tokens.shape
    h = self.llama.tok_embeddings(tokens)
    freqs_cis = self.llama.freqs_cis.to(h.device)
    freqs_cis = freqs_cis[start_pos:start_pos + seqlen]
    mask = None
    mask = torch.full((1, 1, seqlen, seqlen), float("-inf"), device=h.device)
    mask = torch.triu(mask, diagonal=start_pos + 1).type_as(h)


    for layer in self.llama.layers[:-1 * self.query_layer]:
        h = layer(h, start_pos, freqs_cis, mask)
    prefix_query = self.prefix_query.weight.reshape(
        self.query_layer, 1, 4096).unsqueeze(1)
    prefix_index = 0
    
    visual_proj = []
    for i in range(_bsz):
        visual_proj.append(
            visual_feats[i, :, i, :]
        )
    visual_proj = torch.stack(visual_proj)
#     print(visual_proj)
        # B, 1, D
    for layer in self.llama.layers[-1 * self.query_layer:]:
        h = layer(h, start_pos, freqs_cis, mask, visual_proj + prefix_query[prefix_index].repeat(_bsz, 1, 1))
        prefix_index = prefix_index + 1

    h = self.llama.norm(h)
    output = self.llama.output(h[:, -1, :])

    return output.float()


@torch.inference_mode()
def my_generate(
        self,
        inputs,
        prompts,
        max_gen_len: int = 256,
        temperature: float = 0.1,
        top_p: float = 0.75,
        cache_size=10,
        cache_t=20,
        cache_weight=0.5
):
    bsz = len(prompts)
    params = self.llama.params
    assert bsz <= params.max_batch_size, (bsz, params.max_batch_size)

    with torch.cuda.amp.autocast():
        visual_query = self.forward_visual(inputs, cache_size, cache_t, cache_weight)

    if isinstance(prompts[0], str):
        prompts = [self.tokenizer.encode(x, bos=True, eos=False) for x in prompts]

    min_prompt_size = min([len(t) for t in prompts])
    max_prompt_size = max([len(t) for t in prompts])

    total_len = min(params.max_seq_len, max_gen_len + max_prompt_size)

    tokens = torch.full((bsz, total_len), self.tokenizer.pad_id).cuda().long()
    
    reach_eos = torch.zeros(bsz, dtype=torch.bool)
    for k, t in enumerate(prompts):
        tokens[k, : len(t)] = torch.tensor(t).cuda().long()
    input_text_mask = tokens != self.tokenizer.pad_id
    start_pos = min_prompt_size
    prev_pos = 0
    for cur_pos in range(start_pos, total_len):
        with torch.cuda.amp.autocast():
            logits = forward_inference(model, visual_query, tokens[:, prev_pos:cur_pos], prev_pos)
        if temperature > 0:
            probs = torch.softmax(logits / temperature, dim=-1)
            next_token = sample_top_p(probs, top_p)
        else:
            next_token = torch.argmax(logits, dim=-1)
        next_token = next_token.reshape(-1)

        next_token = torch.where(
            input_text_mask[:, cur_pos], tokens[:, cur_pos], next_token
        )
        tokens[:, cur_pos] = next_token
        # trick: early stop if bsz==1
        

        for i in range(bsz):
            if next_token[i] == self.tokenizer.eos_id:
                reach_eos[i] = True
        if (bsz == 1 and next_token[0] == self.tokenizer.eos_id) or (all(reach_eos)):
            break
        
    
        prev_pos = cur_pos

    decoded = []
    for i, t in enumerate(tokens.tolist()):

        # cut to max gen len
        t = t[len(prompts[i]): len(prompts[i]) + max_gen_len]
        # cut to eos tok if any
        try:
            t = t[: t.index(self.tokenizer.eos_id)]
        except ValueError:
            pass
        decoded.append(self.tokenizer.decode(t))

    return decoded

In [7]:
def collate_fn(b):
    batch = {}
    
    audios = []
    prompts = []
    labels = []
    for data in b:
        audio = my_load_and_transform_audio_data(
                        torch.tensor(data["audio"]["array"], dtype=torch.float32
                    ).unsqueeze(0))[0]
        audios.append(audio)
        text = data.get("text").lower() if data.get("text") else None
        instruction = data["instruction"].lower()
        
        prompts.append(llama.format_prompt(instruction, text))
        labels.append(data["label"])
    
    batch["audio"] = torch.stack(audios)
    batch["prompts"] = prompts
    batch["instructions"] = [d["instruction"] for d in b]
    batch["labels"] = [d["label"] for d in b]
    return batch

DATA_PATH = Path("/work/u8915687/big-superb")

In [47]:
all_datasets = [
#     'SpeechBigBench/NoiseDetectionspeech_LJSpeechMusan',
    'SpeechBigBench/AccentClassification_AccentdbExtended', 'SpeechBigBench/BirdSoundDetection_Warblrb10k', 'SpeechBigBench/ChordClassification_AcousticGuitarAndPiano', 'SpeechBigBench/Deeply_Parent_Child_Vocal_Interaction', 'SpeechBigBench/DialogueActClassification_DailyTalk', 'SpeechBigBench/DialogueEmotionClassification_DailyTalk', 'SpeechBigBench/EmotionRecognition_MultimodalEmotionlinesDataset', 'SpeechBigBench/EnhancementDetection_LibrittsTestCleanWham', 'SpeechBigBench/EnvironmentalSoundClassification_AnimalsESC50', 'SpeechBigBench/EnvironmentalSoundClassification_ExteriorAndUrbanNoisesESC50', 'SpeechBigBench/EnvironmentalSoundClassification_HumanAndNonSpeechSoundsESC50', 'SpeechBigBench/EnvironmentalSoundClassification_InteriorAndDomesticSoundsESC50', 'SpeechBigBench/EnvironmentalSoundClassification_NaturalSoundscapesAndWaterSoundsESC50', 'SpeechBigBench/HowFarAreYou_3DSpeaker', 'SpeechBigBench/IntentClassification_FluentSpeechCommands', 'SpeechBigBench/Korean_Read_Speech_Corpus', 'SpeechBigBench/LanguageIdentification_VoxForge', 'SpeechBigBench/NoiseDetectiongaussian_LJSpeechMusan', 'SpeechBigBench/NoiseDetectiongaussian_VCTKMusan', 'SpeechBigBench/NoiseDetectionmusic_LJSpeechMusan', 'SpeechBigBench/NoiseDetectionmusic_VCTKMusan', 'SpeechBigBench/NoiseDetectionnoise_LJSpeechMusan', 'SpeechBigBench/NoiseDetectionnoise_VCTKMusan', 'SpeechBigBench/NoiseDetectionspeech_VCTKMusan', 'SpeechBigBench/NoiseSNRLevelPredictiongaussian_VCTKMusan', 'SpeechBigBench/NoiseSNRLevelPredictionmusic_VCTKMusan', 'SpeechBigBench/NoiseSNRLevelPredictionnoise_VCTKMusan', 'SpeechBigBench/NoiseSNRLevelPredictionspeech_VCTKMusan', 'SpeechBigBench/Nonverbal_Vocalization', 'SpeechBigBench/PronounciationEvaluationAccuracy_Speechocean762', 'SpeechBigBench/PronounciationEvaluationFluency_Speechocean762', 'SpeechBigBench/PronounciationEvaluationOverall_Speechocean762', 'SpeechBigBench/PronounciationEvaluationProsodic_Speechocean762', 'SpeechBigBench/ReverberationDetectionlargeroom_LJSpeechRirsNoises', 'SpeechBigBench/ReverberationDetectionlargeroom_VCTKRirsNoises', 'SpeechBigBench/ReverberationDetectionmediumroom_LJSpeechRirsNoises', 'SpeechBigBench/ReverberationDetectionmediumroom_VCTKRirsNoises', 'SpeechBigBench/ReverberationDetectionsmallroom_LJSpeechRirsNoises', 'SpeechBigBench/ReverberationDetectionsmallroom_VCTKRirsNoises', 'SpeechBigBench/SarcasmDetection_Mustard', 'SpeechBigBench/SpeakerCounting_LibriTTSTestClean', 'SpeechBigBench/SpeechCommandRecognition_GoogleSpeechCommandsV1', 'SpeechBigBench/SpeechDetection_LJSpeech', 'SpeechBigBench/SpeechDetection_LibriSpeechTestClean', 'SpeechBigBench/SpeechDetection_LibriSpeechTestOther', 'SpeechBigBench/SpeechTextMatching_LJSpeech', 'SpeechBigBench/SpeechTextMatching_LibriSpeechTestClean', 'SpeechBigBench/SpeechTextMatching_LibriSpeechTestOther', 'SpeechBigBench/SpokenTermDetection_LJSpeech', 'SpeechBigBench/SpokenTermDetection_LibriSpeechTestClean', 'SpeechBigBench/SpokenTermDetection_LibriSpeechTestOther', 'SpeechBigBench/SpoofDetection_ASVspoof2015', 'SpeechBigBench/SpoofDetection_ASVspoof2017', 'SpeechBigBench/StressDetection_MIRSD', 'SpeechBigBench/arabic_speech_corpus', 'SpeechBigBench/speech_commands']

In [48]:


DATA_PATH = Path("/work/u8915687/big-superb")
ROOT_PATH = Path("/home/u8915687/lab/big-superb/LLaMA-Adapter/imagebind_LLM")
EXP_PATH = ROOT_PATH/ "exp/first"
RESULT_PATH = EXP_PATH/"results3_test"
RESULT_PATH.mkdir(parents=True, exist_ok=True)

for dataset_name in all_datasets:
    print(dataset_name)
    task_name = dataset_name.split("/")[-1]
    
    dataset = load_from_disk(
        DATA_PATH/dataset_name
    )
    
    if not dataset.get("test"):
        print(f"Error {dataset_name}")
        continue
        
    data_loader = torch.utils.data.DataLoader(
        dataset["test"],
        batch_size=16,
        shuffle=False,
        collate_fn=collate_fn
    )
    
    correct = 0
    count = 0
    predictions = []
    label_count = {}
    for batch in tqdm(data_loader):
        inputs = {
            "Audio": [batch["audio"].to("cuda"), 1]
        }
        results = my_generate(
            model,
            inputs,
            batch["prompts"],
            max_gen_len=150,
            temperature=0,
            top_p=0
        )

        
        for r,l, ins in zip(results, batch["labels"], batch["instructions"]):
            r = r.strip()
            if r == l:
                correct += 1
            count += 1
            
            label_count[l] = label_count.get(l, 0) + 1
            predictions.append({
                "pred": r,
                "label": l,
                "instruction": ins
            })
    print(correct, count)
    print(f"{c} / {len(predictions)}")
    json.dump(
        {
            "predictions": predictions,
            "label_count": label_count,
            "accuracy": correct / len(predictions)
        },
        (RESULT_PATH/f"{task_name}.json").open("w"), indent=2
    )
    print("="*100)






SpeechBigBench/AccentClassification_AccentdbExtended


  8%|▊         | 88/1083 [01:57<22:05,  1.33s/it]


KeyboardInterrupt: 

## Single inference

In [141]:
# task = "BigSuperbPrivate/NoiseDetectionSpeech_VoxcelebMusan"
task = "SpeechBigBench/NoiseSNRLevelPredictiongaussian_VCTKMusan"

dataset = load_from_disk(
    DATA_PATH/task
)
dataset = dataset["test"].shuffle(42)[:500]
dataset = Dataset.from_dict(dataset)
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=16,
    shuffle=False,
    collate_fn=collate_fn
)

correct = 0
count = 0
predictions = []
label_count = {}



for batch in tqdm(data_loader):
    inputs = {
        "Audio": [batch["audio"].to("cuda"), 1]
    }
#     prompts = batch["prompts"]
#     prompts = [p.replace(
#         "The answer could be clean or noisy.".lower(), 
#         "The answer could be yes or no.".lower()
#     ) for p in batch["prompts"]]
    prompts = [llama.format_prompt("Determine whether the given audio clip contains real speech or not. The answer could be yes or no.") for p in batch["prompts"]]

    results = my_generate(
        model,
        inputs,
        prompts,
        max_gen_len=150,
        temperature=0,
        top_p=0
    )


    for r,l, ins in zip(results, batch["labels"], batch["instructions"]):
        r = r.strip()
        if r == l:
            correct += 1
        count += 1

        label_count[l] = label_count.get(l, 0) + 1
        predictions.append({
            "pred": r,
            "label": l,
            "instruction": ins
        })

# random score

        
print(prompts[0])
print(correct, count)
print(correct/count)

100%|██████████| 32/32 [00:23<00:00,  1.36it/s]

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Determine whether the given audio clip contains real speech or not. The answer could be yes or no.

### Response:
0 500
0.0


In [142]:
from collections import defaultdict
import random
matrix = defaultdict(dict)

all_labels = [p["label"] for p in predictions]
random.seed(42)
random_count = 0

for p in predictions:
    
    matrix[p["label"]][p["pred"]] = matrix[p["label"]].get(p["pred"], 0) + 1
    if p["label"] == random.choice(all_labels):
        random_count+=1
print(random_count / len(predictions))
print(sorted(matrix.items(), key=lambda x:x[0]))

0.176
[('clean', {'yes': 82, 'no': 3}), ('fifteen', {'yes': 56, 'no': 53}), ('five', {'no': 83, 'yes': 24}), ('ten', {'no': 64, 'yes': 54}), ('zero', {'no': 77, 'yes': 4})]


In [127]:
# new_matrix = defaultdict(dict)
# for k,v in matrix.items():
#     for kk,vv in v.items():
#         if vv >= 2:
#             new_matrix[k][kk] = vv
# matrix = new_matrix

In [143]:
row = set()
column = set()
for k,v in matrix.items():
    row = row.union({k})
    column = column.union(set(v.keys()))
    
row = list(sorted(row))
column = list(sorted(column))

print("pred\label", end=" | ")
for j in range(len(row)):
    print(row[j], end=" | ")
print()
print(end="|")
for j in range(len(row)):
    print("---", end="|")
print()

for i in range(len(column)):
    
    c = column[i]
    print(end="|")
    print(c, end=" | ")
        
    for j in range(len(row)):
        r = row[j]
        
        print(matrix[r].get(c, 0), end=" | ")
    print()
    

pred\label | clean | fifteen | five | ten | zero | 
|---|---|---|---|---|
|no | 3 | 53 | 83 | 64 | 77 | 
|yes | 82 | 56 | 24 | 54 | 4 | 


In [ ]:
import time
audios = []
prompts = []
s1 = time.time()
for i in range(4):
    audio = my_load_and_transform_audio_data(torch.tensor(dataset[i]["audio"]["array"], dtype=torch.float32).unsqueeze(0))[0].unsqueeze(0)
    audio = audio.to("cuda")
    audios.append(audio)
    
    prompts.append(llama.format_prompt(dataset[i]["instruction"]))

    print(dataset[i]["instruction"])
    print("{{",dataset[i]["label"],"}}")
    inputs = {}
    inputs["Audio"] = [audio, 1]
    results = my_generate(
        model,
        inputs,
        [prompts[0]],
        max_gen_len=150,
        temperature=0,
        top_p=0
    )
    print(results[0])
s2 = time.time()
inputs["Audio"] = [torch.cat(audios, dim=0).to("cuda"), 1]
#     inputs["Audio"] = [audio, 1]

print(s2-s1)
results = my_generate(
    model,
    inputs,
    prompts,
    max_gen_len=150,
    temperature=0,
    top_p=0
)
print(results)
print(time.time()-s2)
#     result = results[0].strip()
#     print(result)
#     print(f'{{{dataset[i]["label"]}}}')
#     print("="*100)

In [35]:
f1 = model.forward_visual(
    {"audio":[torch.cat(audios[0:1], dim=0).to("cuda"), 1]}
)

In [36]:
f2 = model.forward_visual(
    {"audio":[torch.cat(audios[1:2], dim=0).to("cuda"), 1]}
)

In [37]:
f1, f2

(tensor([[[[-15.7063,   1.7233,  -3.5650,  ...,  -0.9190,  -3.5915,   0.2461]]]],
        device='cuda:0'),
 tensor([[[[-20.4146,   4.7461,   3.2833,  ..., -10.5704,   2.3532,  -3.9585]]]],
        device='cuda:0'))

In [22]:
f = model.forward_visual(
    {"audio":[torch.cat(audios[0:4], dim=0).to("cuda"), 1]}
)

In [40]:
ff = []
for i in range(4):
    ff.append(f[i, :, i, :])
torch.stack(ff)

tensor([[[-15.7063,   1.7233,  -3.5650,  ...,  -0.9190,  -3.5915,   0.2461]],

        [[-20.4146,   4.7461,   3.2833,  ..., -10.5704,   2.3532,  -3.9585]],

        [[-12.8722,  -4.6297,   2.0977,  ..., -12.4727,   3.0803,   9.3993]],

        [[ -8.4097,  -4.4047,   3.7414,  ...,  -1.2437,   6.6060,   4.4494]]],
       device='cuda:0')

In [55]:
predictions = []
for i in range(len(dataset)):
    batch = dataset[i]
    inputs = {}
    audio = my_load_and_transform_audio_data(torch.tensor(batch["audio"]["array"], dtype=torch.float32).unsqueeze(0))[0].unsqueeze(0)
    audio = audio.to("cuda")
    inputs["Audio"] = [audio, 1]
    
    
#     prompt = llama.format_prompt(batch["instruction"])
    prompt = llama.format_prompt("Does the recording contain recognizable human speech?? The answer could be yes or no.")
    print(prompt)
    
    results = model.generate(
        inputs,
        [prompt],
        max_gen_len=150,
        temperature=0,
        top_p=0
    )
    result = results[0].strip()
    print(result)
    print(f'{{{batch["label"]}}}')
    print("="*100)
    
    predictions.append({
        "pred": result,
        "label": batch["label"],
        "instruction": batch["instruction"]
    })

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Does the recording contain recognizable human speech?? The answer could be yes or no.

### Response:
yes
{clean}
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Does the recording contain recognizable human speech?? The answer could be yes or no.

### Response:
yes
{clean}
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Does the recording contain recognizable human speech?? The answer could be yes or no.

### Response:
yes
{noisy}
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Does the recording contain recognizable human speech?? The answer could be yes or no.

### Response:
no
{noisy}
Below is an instruction that describes a task. Write 

yes
{noisy}
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Does the recording contain recognizable human speech?? The answer could be yes or no.

### Response:
yes
{noisy}
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Does the recording contain recognizable human speech?? The answer could be yes or no.

### Response:
yes
{clean}
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Does the recording contain recognizable human speech?? The answer could be yes or no.

### Response:
yes
{noisy}
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Does the recording contain recognizable human speech?? The answer could be yes or no.

### Response:
yes
{noisy}


In [40]:
c = 0
label_count = {}
for r in predictions:
    if r["pred"] == "yes":
        r["pred"] = "clean"
    if r["pred"] == "no":
        r["pred"] = "noisy"
        
    if r["pred"] == r["label"]:
        c += 1
    
    label_count[r["label"]] = label_count.get(r["label"], 0) + 1
print(c, "/", len(predictions))
print(label_count)

15 / 30
{'clean': 11, 'noisy': 19}


In [73]:
predictions

[]

In [5]:
from datasets import load_from_disk, Audio

d = load_from_disk("/work/u8915687/big-superb/BigSuperbPrivate/SpeakerVerification_Tedlium2Train")

In [7]:
d = d.cast_column("audio", Audio(decode=False))